In [ ]:
import requests
import json
import pandas as pd


api_url = "https://www.agefiph.fr/jsonapi"

services_json = requests.get(api_url + "/node/aide_service")
services_json = services_json.json()
services: list = services_json["data"]

thematiques_json = requests.get(api_url + "/taxonomy_term/thematique")
thematiques_json = thematiques_json.json()
thematiques: list = thematiques_json["data"]

In [ ]:
# with pd.json_normalize the values of fied_essential_* were turned into NaN
# so I switched to a more basic approach
# df_services = pd.json_normalize(services,  sep=".")

services_normalized = []
for service in services:    
    s = {}
    attributes = service["attributes"]
    
    s["id"] = service["id"]
    s["date_creation"] = attributes["created"]
    s["date_maj"] = attributes["changed"]
    s["nom"] = attributes["title"]
    s["presentation_resume"] = attributes["field_titre_card_employeur"]
    
    s["presentation_detail"] = ""
    
    # DESCRIPTIONS
    # TODO: convertir le html des descriptions en markdown
    
    if attributes['field_essentiel_ph']:
        s["presentation_detail"] = f"""Pour la personne handicapée : {attributes['field_essentiel_ph']['processed']}
    """
    if attributes['field_essentiel_employeur']:
        s["presentation_detail"] += f"""Pour l'employeur : {attributes['field_essentiel_employeur']['processed']}"""   
    
    if attributes["field_texte_brut_long"]:
         s["presentation_detail"] = attributes["field_texte_brut_long"] + f"""
         {s["presentation_detail"]}"""   
            
    # THEMATIQUES (seulement les id)
    s["id_thematiques"] = []
    try:
        for thematique in service["relationships"]["field_thematique"]["data"]:
            print(thematique['id'])
            s["id_thematiques"].append(thematique["id"])
    except KeyError:
        print(f"{s['nom']} : pas de thématique")
           
    services_normalized.append(s)  

In [ ]:
thematiques_normalized = {}

for thematique in thematiques:
    attributes = thematique["attributes"]
    print(attributes['name'])
    
    thematiques_normalized[thematique["id"]] =  attributes['name']

In [ ]:
import copy

services_normalized_w_thematiques = []
for service in services_normalized:
    service['thematiques'] = []
    for id_thematique in service["id_thematiques"]:
        service['thematiques'].append(thematiques_normalized[id_thematique])
    del service["id_thematiques"]
    services_normalized_w_thematiques.append(service)
    
pd.DataFrame(services_normalized_w_thematiques).to_csv("agefiph_services_api.csv", index=None)

In [ ]:
# Ajout des données des structures et de contact, récupérées à la main sur le site officiel.